In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [ ]:
dinov2 = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")

In [87]:
from model.dpt_head import create_dpt_head

def map_dinov2_to_croco_naming(net, depth_mode="square", conf_mode="exp"):
    net.enc_embed_dim = net.embed_dim
    net.dec_embed_dim = net.embed_dim
    net.dec_depth = net.n_blocks - 1
    net.depth_mode = depth_mode
    net.conf_mode = conf_mode
    return net

dpt_head = create_dpt_head(
    map_dinov2_to_croco_naming(dinov2),
)
dpt_head.postprocess = None


In [ ]:
from main import get_global_transformer, get_encoder, get_decoder
encoder = get_encoder()
transformer = get_global_transformer(encoder)
decoder = get_decoder(encoder)

In [ ]:
batch_size = 2
img_size = (224, 224)
images = torch.randn(batch_size, 3, *img_size)

tokens_multilayer = encoder.get_intermediate_layers(images, n=encoder.n_blocks)
tokens = tokens_multilayer[-1]

tokens = transformer(tokens)

tokens_multilayer = [tok for tok in tokens_multilayer]
tokens_multilayer[-1] = tokens
preds = decoder(
    tokens_multilayer,
    # [toks.unsqueeze(0) for toks in tokens_multilayer], 
    img_info=img_size,
)
preds.shape

In [ ]:
import torch.distributed as dist
dist.get_world_size()

In [ ]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
def get_global_transformer(encoder):       
    encoder_layer = TransformerEncoderLayer(
        d_model=encoder.embed_dim,  # Match ViT-Base dimension
        nhead=12,
        dim_feedforward=3072,
        dropout=0.1,
        activation='gelu'
    )

    transformer = TransformerEncoder(
        encoder_layer,
        num_layers=6
    )
    return transformer

transformer = get_global_transformer(dinov2)
res = transformer(tokens)

In [ ]:
tokens.shape